In [17]:
# 导入需要的包
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg # mpimg 用于读取图片
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import r2_score
import warnings  
warnings.filterwarnings("ignore")  
%matplotlib inline

# 1--获取数据

In [5]:
def loadData(Dataname):
    Path = 'source/'+str(Dataname)
    TextFile = pd.read_csv(Path)
    return TextFile
trainData = loadData('cs-training.csv')

<img src="source/Image1.png"/>

## 特征缺失情况：
* 月收入和家属数量两个变量出现缺失，分别缺失29731个和3924个

In [6]:
trainData.describe().apply(lambda x:round(x,2))

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.00,150000.00,150000.00,150000.00,150000.00,150000.00,120269.00,150000.00,150000.00,150000.00,150000.00,146076.00
mean,75000.50,0.07,6.05,52.30,0.42,353.01,6670.22,8.45,0.27,1.02,0.24,0.76
std,43301.41,0.25,249.76,14.77,4.19,2037.82,14384.67,5.15,4.17,1.13,4.16,1.12
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,37500.75,0.00,0.03,41.00,0.00,0.18,3400.00,5.00,0.00,0.00,0.00,0.00
50%,75000.50,0.00,0.15,52.00,0.00,0.37,5400.00,8.00,0.00,1.00,0.00,0.00
75%,112500.25,0.00,0.56,63.00,0.00,0.87,8249.00,11.00,0.00,2.00,0.00,1.00
max,150000.00,1.00,50708.00,109.00,98.00,329664.00,3008750.00,58.00,98.00,54.00,98.00,20.00


## 数据预处理----缺失值、异常值、

### 利用随机森林来弥补MonthlyIncome缺失值 ，同时删除 NumberOfDependents变量缺失部分

In [18]:
## 利用随机森林来弥补缺失值 MonthlyIncome 
def fill_missing(df):
    df_monthIncome_missing = df.ix[df['MonthlyIncome'].isnull(),[1,2,3,4,5,7,8,9,10]].values
    df_monthIncome_unmissing = df.ix[df['MonthlyIncome'].notnull(),[1,2,3,4,5,7,8,9,10]].values
    df_monthIncome_Y =  df.ix[df['MonthlyIncome'].notnull(),[6]].values
    # 建立一个训练模型，尽量把误差缩小
    train_X,test_X ,train_Y,test_Y = train_test_split(df_monthIncome_unmissing,df_monthIncome_Y,test_size=0.05,random_state =2018,shuffle =True)
    model_Reg = RandomForestRegressor(random_state=0,n_estimators=100,max_depth  =8,min_samples_split=6,n_jobs=6)
    model_Reg.fit(train_X,train_Y)
    predicted = model_Reg.predict(test_X)
    print("模型的R^2值为：{0}".format(round(r2_score(test_Y,predicted),3))) 
    df.loc[df['MonthlyIncome'].isnull(),['MonthlyIncome']] = model_Reg.predict(df_monthIncome_missing)
    return df

In [19]:
trainData =fill_missing(trainData)

模型的R^2值为：0.71


### 其他部分异常值处理

In [ ]:
# 删除NumberOfDependents变量缺失部分
data = trainData.dropna()
Target_data = data.drop_duplicates()
# 异常值分析
# 年龄大于0
Target_data =Target_data[Target_data['age']>0]
Target_data =Target_data[Target_data['NumberOfTimes90DaysLate']<90]
Target_data['SeriousDlqin2yrs'] = 1-Target_data['SeriousDlqin2yrs']

In [4]:
Target_data = pd.read_csv('Traindata_clean.csv',sep=',',encoding='utf-8')
Target_data.describe().apply(lambda x:round(x,2))

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,145837.00,145837.00,145837.00,145837.00,145837.00,145837.00,145837.00,145837.00,145837.00,145837.00,145837.00
mean,0.93,5.93,52.13,0.25,333.91,5584.29,8.54,0.09,1.03,0.07,0.76
std,0.25,250.28,14.59,0.70,1945.45,13296.49,5.14,0.49,1.13,0.33,1.12
min,0.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,1.00,0.03,41.00,0.00,0.17,2008.00,5.00,0.00,0.00,0.00,0.00
50%,1.00,0.16,52.00,0.00,0.36,4583.00,8.00,0.00,1.00,0.00,0.00
75%,1.00,0.56,62.00,0.00,0.77,7500.00,11.00,0.00,2.00,0.00,1.00
max,1.00,50708.00,107.00,13.00,329664.00,3008750.00,58.00,17.00,54.00,11.00,20.00


In [5]:
# Target_data = Target_data.drop('Unnamed: 0',axis=1)
print(Target_data['SeriousDlqin2yrs'].value_counts())

1    136125
0      9712
Name: SeriousDlqin2yrs, dtype: int64


In [6]:
X,y = Target_data.iloc[:,1:],Target_data['SeriousDlqin2yrs']

In [4]:
X.head(3)

NameError: name 'X' is not defined

### 建立类失衡处理_class,包含三种方法
1. smote函数、RandomOverSampler（两种过采样方法）
2. underSampler（一种欠采样方法）

In [20]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
class class_imbalances:
    def __init__(self,data_X,data_y,ratio,random_state):
        self.X = data_X
        self.y= data_y
        self.ratio = ratio
        self.random_state = random_state
    def class_imbalances_Oversampler_SMOTE(self):
        '''SMOTE过采样即对少类样本进行smote函数生成成指定N个少类样本集，N个少数样本和全部的多类样本组成新的数据集'''
        X_resampled,y_resampled = SMOTE(ratio = self.ratio).fit_sample(self.X,self.y)
        print("源数据y的分布",sorted(Counter(self.y).items()))
        print("SMOTE过采样源数据y的分布",sorted(Counter(y_resampled).items()))
        return (X_resampled,y_resampled)
    def class_imbalances_RandomOverSample(self):
        ''' 过采样即从少类样本中复制 N*[少类样本个数]个数，和全部的多类样本组成新的数据集'''
        ros = RandomOverSampler(random_state=2018,ratio=self.ratio)
        X_resampled,y_resampled = ros.fit_sample(self.X,self.y)
        print("源数据y的分布",sorted(Counter(self.y).items()))
        print("RandomOverSample过采样源数据y的分布",sorted(Counter(y_resampled).items()))
        return (X_resampled,y_resampled)
    def class_imbalances_UnderSample(self):
        ''' 欠采样即从多类样本中挑选N*[少类样本个数]个多类样本个数，和全部的少类样本组成新的数据集'''
        RUS =  RandomUnderSampler(ratio=self.ratio,random_state=2018)
        X_resampled,y_resampled = RUS.fit_sample(self.X,self.y)
        print("源数据y的分布",sorted(Counter(self.y).items()))
        print("Undersample欠采样源数据y的分布",sorted(Counter(y_resampled).items()))
        return (X_resampled,y_resampled)

## 类失衡处理后数据分布情况

In [11]:
imbalance = class_imbalances(data_X=X,data_y=y,random_state=2018,ratio=0.3)
X_resampled,y_resampled = imbalance.class_imbalances_Oversampler_SMOTE()

源数据y的分布 [(0, 9712), (1, 136125)]
SMOTE过采样源数据y的分布 [(0, 40837), (1, 136125)]


In [37]:
# imbalance = class_imbalances(data_X=X,data_y=y,random_state=2018,ratio=0.3)
# X_resampled,y_resampled = imbalance.class_imbalances_RandomOverSample()

源数据y的分布 [(0, 9712), (1, 136125)]
RandomOverSample过采样源数据y的分布 [(0, 40837), (1, 136125)]


In [44]:
# imbalance = class_imbalances(data_X=X,data_y=y,random_state=2018,ratio=0.2)
# X_resampled,y_resampled = imbalance.class_imbalances_UnderSample()

源数据y的分布 [(0, 9712), (1, 136125)]
Undersample欠采样源数据y的分布 [(0, 9712), (1, 48560)]


## 三种变量选择的方法
1. selectBest： 是一种基于p-values值的是否显著性来进行排序，显著则保留top K个
2. selectFromModel 方法 是一种基于变量的feature_importance 来进行判断，大于threshold则保留，否则放弃
3. 利用LogisticsRegression回归中的逐步回来来筛选变量

In [44]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
class class_FeatureSelect:
    def __init__(self,data_X,data_y,random_state,n_jobs=4):
        self.X = data_X
        self.y= data_y
        self.random_state = random_state
        self.start_time = datetime.now()
        self.njobs =n_jobs
    def Select_Feature(self):
        score_estimator ={}
        clf =LogisticRegressionCV(cv=5,penalty='l2',scoring=None,random_state=self.random_state, solver='lbfgs')
        for i in range(1,self.X.shape[1]+1):
            # 建立变量选择模型
            SK = SelectKBest(k=i)
            # 获取指定数量的特征
            X_new =SK.fit_transform(self.X,self.y)
            # 建立分类模型进行评估
            clf.fit(X_new,self.y)
            score =clf.score(X_new,self.y)
            score_estimator[i] =score
        # 模型得分进行排序取最大值的特征个数
        best_FeatureNum =sorted(score_estimator.items(),key=lambda x:x[1],reverse= True)[0][0]
        SK = SelectKBest(k=best_FeatureNum)
        # 获取指定最优数量的特征
        X_Best =SK.fit_transform(self.X,self.y)
        clf.fit(X_Best,self.y)
        print("原始数据集的变量个数{0}".format(self.X.shape[1]))
        print("SelectKBest筛选后数据集的变量个数{0}".format(X_Best.shape[1]))
        print("SelectKbest模型的得分是:",clf.score(X_Best,self.y))
        print("total process cost time is {0}".format(datetime.now()-self.start_time))
        return X_Best
    def Select_FromModel(self):
        accuracy_score ={}
        model = RandomForestClassifier(n_estimators=100,random_state=self.random_state,n_jobs =  self.njobs)
        for i in range(1,13):
            # 模型得分进行排序取最大值的特征个数
            sfm = SelectFromModel(model,threshold="{0}*mean".format(float(i*0.1)))
            # 获取指定最优数量的特征
            X_new =sfm.fit_transform(self.X,self.y)
            X_train, X_test, Y_trian, Y_test = train_test_split(X_new,self.y,test_size=0.2,stratify=self.y)
            model.fit(X_train, Y_trian)
            predicted = model.predict(X_test)
            print("i ={0}时准确率为：{1}".format(i,sum(predicted == Y_test)/len(Y_test)))
            accuracy_score[i] = sum(predicted == Y_test)/len(Y_test)
        # 模型得分进行排序取最大值的特征个数
        best_Num =sorted(accuracy_score.items(),key=lambda x:x[1],reverse= True)[0][0]
        sfm = SelectFromModel(model,threshold="{0}*mean".format(float(best_Num*0.1)))
        # 获取指定最优数量的特征
        X_Best =sfm.fit_transform(self.X,self.y)
        model.fit(X_Best,self.y)
        print("原始数据集的变量个数{0}".format(self.X.shape[1]))
        print("SelectKBest筛选后数据集的变量个数{0} and threshold is {1}*mean".format(X_Best.shape[1],best_Num))
        print("SelectKbest模型的得分是:",accuracy_score[best_Num])
        print("total process cost time is {0}".format(datetime.now()-self.start_time))
        return X_Best

In [45]:
FeatureSelect  = class_FeatureSelect(data_X = X_resampled,data_y=y_resampled,random_state=2018,n_jobs=6)

In [46]:
X_best_select =FeatureSelect.Select_FromModel()

i =1时准确率为：0.9430113299239963
i =2时准确率为：0.9453846805865567
i =3时准确率为：0.9442827677789394
i =4时准确率为：0.9445370553499279
i =5时准确率为：0.9420789421637047
i =6时准确率为：0.9370214449184867
i =7时准确率为：0.9061678863052016
i =8时准确率为：0.8825473963778149
i =9时准确率为：0.8811911959992089
i =10时准确率为：0.9192213149492837
i =11时准确率为：0.9126946006272427
i =12时准确率为：0.9129771423727856
原始数据集的变量个数10
SelectKBest筛选后数据集的变量个数10 and threshold is 2*mean
SelectKbest模型的得分是: 0.9453846805865567
total process cost time is 0:03:36.502325


In [96]:
# 数据集保存
colname =list(Target_data.columns)
data_Smote =pd.DataFrame(np.concatenate((y_resampled.reshape(-1,1),X_best_select),axis=1),columns=colname)

In [97]:
data_Smote.head(2)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0.0,0.766127,45.0,2.0,0.802982,9120.0,13.0,0.0,6.0,0.0,2.0
1,1.0,0.957151,40.0,0.0,0.121876,2600.0,4.0,0.0,0.0,0.0,1.0


### 数据集保存

In [49]:
# data_Smote.to_csv('data_smote.csv',header=True,index = False,encoding='utf-8')
data_Smote = pd.read_csv('Traindata_clean.csv')
data_Smote.head(3)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,1,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0


# 针对数据源WOE转换并替换原数据集

In [21]:
class class_Preprocess_WOE_Reshape:
    def __init__(self,data,bins):
        self.Data = data
        self.bins = bins
    def calcu_woe(self):
        woe_dict = {}
        for col in self.Data.columns.tolist(): 
            woe_List =[]
            if col=='SeriousDlqin2yrs':
                Allgood = self.Data[col].value_counts()[1]
                Allbad = self.Data[col].value_counts()[0]
            else:
                for i in np.arange(len(self.bins[col])-1):
                    Min =self.Data[col]<=self.bins[col][i+1]
                    Max =self.Data[col]>self.bins[col][i]
                    good = self.Data.loc[(Min&Max),'SeriousDlqin2yrs'].sum()
                    bad = len(self.Data.loc[(Min&Max),'SeriousDlqin2yrs'])-good
                    woe = np.log((good/Allgood)*(Allbad/bad))
                    woe_List.append(round(woe,4))
                woe_dict[col] =woe_List
        return woe_dict
    def Data_Preprocessing_Cut(self):
        Data_Cut = pd.DataFrame()
        for col in list( self.Data.columns[1:]):
            Data_Cut[col] = pd.cut( self.Data[col],bins =self.bins[col]).map(str).tolist()
        return Data_Cut
    def data_Preprocessing_Woe_Replace(self):
        # first step . use woe values and binning make dict for replace
        Data = self.Data.copy()
        for col in Data.columns.tolist()[1:]:
            woe_bins_dict ={}
            woe_dict = calcu_woe(Data=self.Data,bins_seq=self.bins)
            Data_Smote_Cut = Data_Preprocessing_Cut(Data=self.Data,bins_seq=self.bins)
            bins_sort_list = pd.cut(Data[col],bins =self.bins[col]).unique().sort_values().tolist()
            for i,bins in enumerate(bins_sort_list):
                woe_bins_dict[str(bins)] = woe_dict[col][i]
            Data[col] = Data_Smote_Cut[col].map(woe_bins_dict)
        return Data

### 读取WOE转换后数据

In [22]:
Target_data_woechange = pd.read_csv('Data_Smote_Cut_replace.csv',encoding='utf-8')

In [23]:
Target_data_woechange[:3]

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,-1.0818,-0.3553,-1.7379,-0.0005,0.4113,0.1242,0.3751,-0.2586,0.2729,-0.2373
1,1,-1.0818,-0.3553,0.5275,-0.0005,-0.3321,0.0250,0.3751,-0.0232,0.2729,-0.0996
2,1,-0.1158,-0.3553,-0.9063,-0.0005,-0.2824,-0.2559,-1.9657,-0.0232,0.2729,0.1553


# 建立logisticsRegression模型

### 数据集准备

In [24]:
X_Data = Target_data_woechange.iloc[:,1:]
X_Data[:3]

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,-1.0818,-0.3553,-1.7379,-0.0005,0.4113,0.1242,0.3751,-0.2586,0.2729,-0.2373
1,-1.0818,-0.3553,0.5275,-0.0005,-0.3321,0.0250,0.3751,-0.0232,0.2729,-0.0996
2,-0.1158,-0.3553,-0.9063,-0.0005,-0.2824,-0.2559,-1.9657,-0.0232,0.2729,0.1553


In [28]:
X_Data.shape

(145837, 10)

In [25]:
y_Data = Target_data_woechange['SeriousDlqin2yrs'].values
y_Data[:2]

array([0, 1], dtype=int64)

## 利用类失衡进行处理

In [26]:
imbalance =class_imbalances(data_X =X_Data,data_y= y_Data,ratio=0.5,random_state=2018)

In [27]:
X_oversampler,Y_oversampler = imbalance.class_imbalances_Oversampler_SMOTE()

源数据y的分布 [(0, 9712), (1, 136125)]
SMOTE过采样源数据y的分布 [(0, 68062), (1, 136125)]


## 模型参数选择

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
grid_param ={'max_iter':[100,200,500],'C':[0.01,0.1,1,10,100],'solver':['liblinear','lbfgs']}
def Select_bestParameter(data_X,data_Y,grid_prarm):
    clf = LogisticRegression(penalty="l2",random_state=2018)
    grid = GridSearchCV(clf,grid_param,cv=5,n_jobs=6)
    grid.fit(data_X,data_Y)
    return grid

In [31]:
gird =Select_bestParameter(X_oversampler,Y_oversampler,grid_param)

In [32]:
gird.best_params_

{'C': 0.1, 'max_iter': 100, 'solver': 'liblinear'}

# 模型评估

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,roc_auc_score
X_train,X_test,Y_train,Y_test = train_test_split(X_oversampler,Y_oversampler,test_size=0.2,random_state = 2018,stratify =Y_oversampler)
# best_model =gird.best_estimator_
best_model = LogisticRegression(penalty="l2",random_state=2018,C=0.1,max_iter=100,solver='liblinear')
best_model.fit(X_train,Y_train)
predicted = best_model.predict(X_test)
predicted_prob = best_model.predict_proba(X_test)

# f1_score值输出

In [35]:
f1_score(Y_test,predicted)

0.8609339996553507

## roc_auc_score 值输出

In [36]:
score =predicted_prob[:,1:]
score =score.ravel()
roc_auc_score(Y_test,score)

0.8532501119595006

In [37]:
best_model.coef_

array([[0.63964474, 0.51062076, 0.60386648, 0.74452679, 0.38330059,
        0.03230222, 0.61346855, 0.74766174, 0.49945693, 0.13514052]])

In [65]:
coef_list =best_model.coef_.tolist()[0]
coef_list

[0.6396447416115161,
 0.5106207550941492,
 0.6038664811482479,
 0.7445267856937159,
 0.38330058790445226,
 0.03230222355206056,
 0.6134685490470583,
 0.7476617448394686,
 0.4994569303921842,
 0.13514052055078254]

In [96]:
best_model.intercept_

array([0.66164068])

# 最终评分卡模型

In [44]:
bins_ROUL =[-np.inf,0,0.3,0.7,1,3,np.inf]  # RevolvingUtilizationOfUnsecuredLines
bins_age =[-np.inf,30,45,55,60,65,75,np.inf]  # age
bins_N359DW = [-np.inf,0,1,3,5,np.inf]  # NumberOfTime30-59DaysPastDueNotWorse
bins_Ratio = [-np.inf,0,1,3,5,10,np.inf]   # DebtRatio
bins_moncome =[-np.inf,751,3000,4500,6100,8850,np.inf] # MonthlyIncome
bins_NCLL =[-np.inf,0,1,3,5,np.inf] # NumberOfOpenCreditLinesAndLoans
bins_N90DL = [-np.inf,0,1,2,5,np.inf]   # NumberOfTimes90DaysLate
bins_NRLL = [-np.inf,1,2,np.inf]   # NumberRealEstateLoansOrLines
bins_N689DW = [-np.inf,0,1,3,5,np.inf]  # NumberOfTime60-89DaysPastDueNotWorse
bins_NOD =[-np.inf,0,1,3,5,np.inf] # NumberOfDependents
bins_seq ={}
for name,bins in zip(list(Target_data_woechange.columns)[1:],[bins_ROUL,bins_age,bins_N359DW,bins_Ratio,bins_moncome,bins_NCLL,bins_N90DL,bins_NRLL,bins_N689DW,bins_NOD]):
    bins_seq[name] = bins

In [45]:
bins_seq

{'RevolvingUtilizationOfUnsecuredLines': [-inf, 0, 0.3, 0.7, 1, 3, inf],
 'age': [-inf, 30, 45, 55, 60, 65, 75, inf],
 'NumberOfTime30-59DaysPastDueNotWorse': [-inf, 0, 1, 3, 5, inf],
 'DebtRatio': [-inf, 0, 1, 3, 5, 10, inf],
 'MonthlyIncome': [-inf, 751, 3000, 4500, 6100, 8850, inf],
 'NumberOfOpenCreditLinesAndLoans': [-inf, 0, 1, 3, 5, inf],
 'NumberOfTimes90DaysLate': [-inf, 0, 1, 2, 5, inf],
 'NumberRealEstateLoansOrLines': [-inf, 1, 2, inf],
 'NumberOfTime60-89DaysPastDueNotWorse': [-inf, 0, 1, 3, 5, inf],
 'NumberOfDependents': [-inf, 0, 1, 3, 5, inf]}

In [89]:
WOE_Reshape = class_Preprocess_WOE_Reshape(data = data_Smote,bins=bins_seq)
woe_dict = WOE_Reshape.calcu_woe()
woe_dict

{'RevolvingUtilizationOfUnsecuredLines': [0.8288,
  1.1802,
  -0.1158,
  -1.0818,
  -2.2233,
  -0.4941],
 'age': [-0.5688, -0.3553, -0.1361, 0.2853, 0.5439, 0.9594, 1.1947],
 'NumberOfTime30-59DaysPastDueNotWorse': [0.5275,
  -0.9063,
  -1.7379,
  -2.3839,
  -2.6572],
 'DebtRatio': [0.0129, -0.0005, -0.7209, 0.0224, 0.4969, 0.1871],
 'MonthlyIncome': [0.2474, -0.3321, -0.2824, -0.092, 0.1839, 0.4113],
 'NumberOfOpenCreditLinesAndLoans': [-1.3749, -0.7978, -0.2559, 0.025, 0.1242],
 'NumberOfTimes90DaysLate': [0.3751, -1.9657, -2.6376, -3.0776, -3.3085],
 'NumberRealEstateLoansOrLines': [-0.0232, 0.1844, -0.2586],
 'NumberOfTime60-89DaysPastDueNotWorse': [0.2729,
  -1.833,
  -2.6953,
  -3.1007,
  -3.3874],
 'NumberOfDependents': [0.1553, -0.0996, -0.2373, -0.4537, -0.7082]}

In [67]:
coef_dict =dict(zip(X_Data.columns.tolist(),coef_list))

In [71]:
coef_dict

{'RevolvingUtilizationOfUnsecuredLines': 0.6396447416115161,
 'age': 0.5106207550941492,
 'NumberOfTime30-59DaysPastDueNotWorse': 0.6038664811482479,
 'DebtRatio': 0.7445267856937159,
 'MonthlyIncome': 0.38330058790445226,
 'NumberOfOpenCreditLinesAndLoans': 0.03230222355206056,
 'NumberOfTimes90DaysLate': 0.6134685490470583,
 'NumberRealEstateLoansOrLines': 0.7476617448394686,
 'NumberOfTime60-89DaysPastDueNotWorse': 0.4994569303921842,
 'NumberOfDependents': 0.13514052055078254}

In [70]:
bins_seq

{'RevolvingUtilizationOfUnsecuredLines': [-inf, 0, 0.3, 0.7, 1, 3, inf],
 'age': [-inf, 30, 45, 55, 60, 65, 75, inf],
 'NumberOfTime30-59DaysPastDueNotWorse': [-inf, 0, 1, 3, 5, inf],
 'DebtRatio': [-inf, 0, 1, 3, 5, 10, inf],
 'MonthlyIncome': [-inf, 751, 3000, 4500, 6100, 8850, inf],
 'NumberOfOpenCreditLinesAndLoans': [-inf, 0, 1, 3, 5, inf],
 'NumberOfTimes90DaysLate': [-inf, 0, 1, 2, 5, inf],
 'NumberRealEstateLoansOrLines': [-inf, 1, 2, inf],
 'NumberOfTime60-89DaysPastDueNotWorse': [-inf, 0, 1, 3, 5, inf],
 'NumberOfDependents': [-inf, 0, 1, 3, 5, inf]}

# 最后Score表达式

In [82]:
A = 541.85
B = -28.86

In [93]:
woe_score_dict =woe_dict.copy()
for col,item in woe_dict.items():
    for i,woe in enumerate(item):
        woe_score_dict[col][i] = woe*coef_dict[col]*B

In [85]:
coef_dict

{'RevolvingUtilizationOfUnsecuredLines': 0.6396447416115161,
 'age': 0.5106207550941492,
 'NumberOfTime30-59DaysPastDueNotWorse': 0.6038664811482479,
 'DebtRatio': 0.7445267856937159,
 'MonthlyIncome': 0.38330058790445226,
 'NumberOfOpenCreditLinesAndLoans': 0.03230222355206056,
 'NumberOfTimes90DaysLate': 0.6134685490470583,
 'NumberRealEstateLoansOrLines': 0.7476617448394686,
 'NumberOfTime60-89DaysPastDueNotWorse': 0.4994569303921842,
 'NumberOfDependents': 0.13514052055078254}

In [92]:
woe_score_dict

{'RevolvingUtilizationOfUnsecuredLines': [0.8288,
  1.1802,
  -0.1158,
  -1.0818,
  -2.2233,
  -0.4941],
 'age': [-0.5688, -0.3553, -0.1361, 0.2853, 0.5439, 0.9594, 1.1947],
 'NumberOfTime30-59DaysPastDueNotWorse': [0.5275,
  -0.9063,
  -1.7379,
  -2.3839,
  -2.6572],
 'DebtRatio': [0.0129, -0.0005, -0.7209, 0.0224, 0.4969, 0.1871],
 'MonthlyIncome': [0.2474, -0.3321, -0.2824, -0.092, 0.1839, 0.4113],
 'NumberOfOpenCreditLinesAndLoans': [-1.3749, -0.7978, -0.2559, 0.025, 0.1242],
 'NumberOfTimes90DaysLate': [0.3751, -1.9657, -2.6376, -3.0776, -3.3085],
 'NumberRealEstateLoansOrLines': [-0.0232, 0.1844, -0.2586],
 'NumberOfTime60-89DaysPastDueNotWorse': [0.2729,
  -1.833,
  -2.6953,
  -3.1007,
  -3.3874],
 'NumberOfDependents': [0.1553, -0.0996, -0.2373, -0.4537, -0.7082]}